In [ ]:
def load_TVT(divide_flag,file_locate):
    if divide_flag == False:
        train_img=np.load(file_locate+'train_img.npy');
        train_lab=np.load(file_locate+'train_lab.npy');
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');

        print "Training Data Image",np.shape(train_img)
        print "Training Data Label",np.shape(train_lab)
        print "Test Data Label",np.shape(test_lab)
        print "Test Data Image",np.shape(test_img)
        print "val Data Label" , np.shape(val_lab)
        print "val Data Image" , np.shape(val_img)
        
        n_train= np.shape(train_img)[0]
        n_train_lab = np.shape(train_lab)[0]
        return train_img ,train_lab,val_img,val_lab,test_img,test_lab
   
    if divide_flag == True:
        print '트레이닝 파일이 여러개로 분할되어 있습니다. 분할된 트레이닝 파일에 대한 조치가 필요합니다'
        train_images, train_labels =get_batch_list(file_locate)
        print train_images ,train_labels
        val_img= np.load(file_locate+'val_img.npy');
        val_lab = np.load(file_locate+'val_lab.npy');
        test_img=np.load(file_locate+'test_img.npy');
        test_lab=np.load(file_locate+'test_lab.npy');
        print "the number of training image batch",len(train_images)
        print "the number of training label batch",len(train_labels)
        print "Test Data Label",np.shape(test_lab)
        print "Test Data Image",np.shape(test_img)
        print "val Data Label" , np.shape(val_lab)
        print "val Data Image" , np.shape(val_img)
        return train_images, train_labels,val_img,val_lab,test_img,test_lab

In [ ]:
def aug_crop(np_img ,img_row ,img_col , color_ch, crop_img_row , crop_img_col , label):
    n_img = np.shape(np_img)[0] 
    n_ret_img = n_img*(img_row - crop_img_row) * (img_col - crop_img_col)*2
    ret_images = np.zeros([n_ret_img ,crop_img_row , crop_img_col,color_ch])
    len_label= np.shape(label)[1]
    ret_labels =  np.zeros([n_ret_img  ,len_label])
    #print "n_augmented image size : " , n_ret_img 
    #print "n classes :", len_label
    #copy label in factor by 2014

    if len(np.shape(np_img))==2:
        np_img=np.reshape(np_img , newshape = [np.shape(np_img)[0] , img_row , img_col ,color_ch])
        print np.shape(np_img)
    for n  in range(len(np_img)):
        ret_labels[n*2 , : ] = label[n,:]
        ret_labels[n*2+1 , : ] = label[n,:]
        for r in range(img_row - crop_img_row):
            for c in range(img_col - crop_img_col):

                cropped_img = np_img[n, r:crop_img_row +r , c:crop_img_col+c ,: ]

                ret_images[n*2,:,:,:]=cropped_img  
                ret_images[(n*2+1) , :,:,:] =np.fliplr(cropped_img )


    return ret_images ,ret_labels



In [ ]:
def aug_8_times(x):
    """
    
    
    x shape is [n_batch , row ,col , color_ch ]
    x type is numpy 
    this code need too many time to run 
    we should find solution using parallel method to less run time maybe 
    
    return x,np_rot90,np_rot180,np_rot270,lr_x,np_lr_rot90 ,np_lr_rot180 , np_lr_rot270 
    
    """
    start_time =time.time()
    n_batch,row,col,ch=np.shape(x)
    lr_x = np.flipud(x)

    np_rot90 =np.zeros(shape =[n_batch , row ,col ,ch] )
    np_rot180=np.zeros(shape =[n_batch , row ,col ,ch] )
    np_rot270=np.zeros(shape =[n_batch , row ,col ,ch] )

    np_lr_rot90 =np.zeros(shape =[n_batch , row ,col ,ch] )
    np_lr_rot180=np.zeros(shape =[n_batch , row ,col ,ch] )
    np_lr_rot270=np.zeros(shape =[n_batch , row ,col ,ch] )
    
    

    for batch_ind in range(n_batch):

        rot90=np.rot90(x[batch_ind,:,:,:])
        rot180=np.rot90(rot90)
        rot270=np.rot90(rot180)

        np_rot90[batch_ind,:,:,:] = rot90
        np_rot180[batch_ind,:,:,:]=rot180
        np_rot270[batch_ind,:,:,:]=rot270

        lr_rot90=np.rot90(lr_x[batch_ind,:,:,:])
        lr_rot180=np.rot90(lr_rot90)
        lr_rot270=np.rot90(lr_rot180)

        np_lr_rot90[batch_ind,:,:,:]=lr_rot90
        np_lr_rot180[batch_ind,:,:,:]=lr_rot180
        np_lr_rot90[batch_ind,:,:,:]=lr_rot270
    end_time =time.time()
    print end_time - start_time
    return x,np_rot90,np_rot180,np_rot270,lr_x,np_lr_rot90 ,np_lr_rot180 , np_lr_rot270 
    

In [ ]:

def training(batch_xs , batch_ys):
    sess.run(train_step ,feed_dict={x: batch_xs, y_:batch_ys , keep_prob : 0.7})

In [ ]:
with tf.device('/gpu:0'):
    def next_batch(batch_size , image , label):

        a=np.random.randint(np.shape(image)[0] -batch_size)
        batch_x = image[a:a+batch_size,:]
        batch_y= label[a:a+batch_size,:]
        return batch_x, batch_y


In [ ]:
#dirname = '/home/ncc/notebook/mammo/result/'
def make_logdir(dirname):
  

    count=0
    while(True):
        if not os.path.isdir(dirname):
            os.mkdir(dirname)
            break
        elif not os.path.isdir(dirname + str(count)):
            dirname=dirname+str(count)
            os.mkdir(dirname)
            break
        else:
            count+=1
    print 'it is recorded at :'+str(count)

    f=open(dirname+"/log.txt",'w')
    return f

In [ ]:

def write_log(step,train_acc, train_loss , val_acc , val_loss ,fp):
    """
    fp = File Pointer
    
    """
    #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
    print step
    print("step %d , training  accuracy %g" %(step,train_acc))
    print("step %d , loss : %g" %(step,train_loss))
    train_str = 'step:\t'+str(step)+'\tval_loss:\t'+str(train_loss) +'\tval accuracy:\t'+str(train_acc)+'\n'

    print("step %d , validation  accuracy %g" %(step,val_acc))
    print("step %d , validation loss : %g" %(step,val_loss))
    val_str = 'step:\t'+str(step)+'\tval_loss:\t'+str(val_loss) +'\tval accuracy:\t'+str(val_acc)+'\n'

    fp.write(train_str)
    fp.write(val_str)
    

In [ ]:
def validate_from_images(sess, images , labels , accuracy , cost ):
    """
    input : x-
    
    x type  : numpy 
    x shape : [n , row , col , ch]
    return  acc,  loss

    """
    print accuracy
    acc_list=[]
    loss_list=[]
    images_labels=zip(images,labels)
    for img_ind ,(img,lab)  in enumerate(images_labels):

        acc ,loss = sess.run( [accuracy,cost] ,feed_dict={x:img , y_: lab , keep_prob: 1.0})        
        acc_list.append(acc)
        loss_list.append(loss)

    acc_list=np.asarray(acc_list)
    loss_list=np.asarray(loss_list)
    acc=np.mean(acc_list)
    loss=np.mean(loss_list)
    
    return  acc,  loss


In [ ]:
def validate(img , lab ,accuracy ,cost ): #default
    """
    return val_acc,  val_loss, train_acc, train_loss
    """    
    acc,loss = sess.run([accuracy,cost] , feed_dict={x:img , y_:lab , keep_prob: 1.0})        
    return acc,loss

In [ ]:

def divide_images(images , labels,batch_size):
    """
    return list_images,list_labels
    """
    n_divide=len(images)/batch_size

    list_images=[]
    list_labels=[]
    for ind in range(n_divide):
        # j*batch_size :(j+1)*batch_size
        image =images[ ind*batch_size :(ind+1)*batch_size] 
        label =labels[ ind*batch_size :(ind+1)*batch_size]
        list_images.append(image)
        list_labels.append(label)

    #right above code have to modify
    image = images[ (ind+1)*batch_size :  ] 
    label = labels[ (ind+1)*batch_size :  ]
    list_images.append(image)
    list_labels.append(label)

    return list_images,list_labels

In [ ]:
  
def validate_extract_imgs(val_img , val_lab , train_img , train_lab ):
    """
    extract patch from ori-image
    """
    color_ch = in_ch
    val_images  =extract_test_img(val_img , 128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
    train_images=extract_test_img(train_img ,128 , 128 , color_ch   ,crop_img_row =118 , crop_img_col =118 )
    val_acc, val_loss =validate_from_images(sess, val_images , val_lab , accuracy ,cost )
    train_acc , train_loss =validate_from_images(sess, train_images , train_lab ,accuracy , cost)
    
    return val_acc , val_loss, train_acc ,train_loss
    


In [ ]:
def BATCH_TRAINING_MANUAL(maxiter  , batch_size,file_locate, cost , train_step ,correct_prediction , \
                   accuracy , fp ):
    """
    BATCH : 이 함수는 이미지를 한장씩 처리하는게 아닌 여러장을 묶어(batch) 처리 할 수 있게 합니다
    MANUAL:이 함수는 Training batch파일을 순서대로 training 합니다.
    AUG: 이 함수는 8배짜리 augmentation을 하고 트레이닝을 합니다.  
    
    
    val_acc, val_loss =validate_from_images(val_imgs ,val_labs ,accuracy, cost)
    여기 부분 검증해야 한다.            
    """
    start_time = time.time()
#with tf.device('/gpu:0'):
    train_img, train_lab,val_img,val_lab,test_img,test_lab\
    =load_TVT(divide_flag,file_locate)
    train_images , train_labels=divide_images(train_img , train_lab , batch_size)
    train_images_labels=zip(train_images , train_labels)
    
    for step,(batch_xs , batch_ys) in train_images_labels:
        if step%100 ==0:
            try:
                val_acc , val_loss=validate(val_img , val_lab)
                train_acc , train_loss=validate(batch_xs,batch_ys)
            except:
                #한번에 트레이닝이 안되면 분할해서 validation한다.
                val_imgs, val_labs = divide_images(val_img , val_lab, batch_size) #
                val_acc, val_loss =validate_from_images(val_imgs ,val_labs ,accuracy, cost)
                train_acc , train_loss=validate(batch_xs,batch_ys,accuracy, cost)
            write_log(step , train_acc, train_loss , val_acc, val_loss , fp)
            #np.save('./result/batch_xs' ,batch_xs)
            #np.save('./result/batch_ys' ,batch_ys)
        else:
            training(batch_xs ,batch_ys)                    


############################################aug_and_training############################
    print("--- Training Time : %s ---" % (time.time() - start_time))
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    fp.write(train_time)
    fp.close()

In [ ]:
def BATCH_TRAINING_MANUAL_AUG8(maxiter, batch_size,file_locate, tensor_info, fp ):
 """
  BATCH : 이 함수는 이미지를 한장씩 처리하는게 아닌 여러장을 묶어(batch) 처리 할 수 있게 합니다
  MANUAL:이 함수는 Training batch파일을 순서대로 training 합니다.
  AUG: 이 함수는 8배짜리 augmentation을 하고 트레이닝을 합니다.  
  
  to do: validation을 마지막에 학습시키는 코드를 추가해야 합니다.
  
  
 """
    cost,train_step,correct_prediction , accuracy , softmax = tensor_info
    start_time = time.time()
#with tf.device('/gpu:0'):
    train_img, train_lab,val_img,val_lab,test_img,test_lab\
    =load_TVT(False,file_locate) #TVT is Train Validation Test
    train_imgs, train_labs = divide_images(train_img , train_val, batch_size)
    for step in range(len(train_imgs)*maxiter)
        
        if step > len(train_imgs):
            ind =0
        batch_xs , batch_ys = train_imgs[ind] , train_labs[ind]
        ind+=1
        
        list_aug_x=aug_8_times(batch_xs)
        for ele in list_aug_x:
            training(ele,batch_ys)
        
        if step%100 ==0: #여기 if loop에서 validate을 합니다 
            try:
                val_acc , val_loss=validate(val_img , val_lab)
                train_acc , train_loss=validate(batch_xs,batch_ys)
            except:
                #한번에 트레이닝이 안되면 분할해서 validation한다.
                val_imgs, val_labs = divide_images(val_img , val_lab, batch_size)
                print np.shape(val_imgs[0])
                val_acc, val_loss =validate_from_images(sess,val_imgs ,val_labs ,accuracy ,cost )
                train_acc , train_loss=validate(batch_xs,batch_ys,accuracy ,cost)

            write_log(step , train_acc, train_loss , val_acc, val_loss , fp)
            softmax_=sess.run(softmax ,feed_dict ={ x:batch_xs , y_:batch_ys , keep_prob:1.0 })
            print softmax_
            
            
        

############################################aug_and_training############################
    print("--- Training Time : %s ---" % (time.time() - start_time))
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    fp.write(train_time)
    fp.close()

In [ ]:
def BATCH_TRAINING_RANDOM_AUG8(maxiter, batch_size,file_locate, cost , train_step ,correct_prediction , \
                   accuracy ,softmax, fp ):
 """
 aug( O )
 
 
 """ 
    cost,train_step,correct_prediction , accuracy , softmax = tensor_info
    start_time = time.time()
#with tf.device('/gpu:0'):
    train_img, train_lab,val_img,val_lab,test_img,test_lab\
    =load_TVT(False,file_locate) #TVT is Train Validation Test  
    for step in range(maxiter):    
        batch_xs , batch_ys = next_batch(batch_size, train_img , train_lab)       
        if step%100 ==0: #여기 if loop에서 validate을 합니다 
            try:
                val_acc , val_loss=validate(val_img , val_lab)
                train_acc , train_loss=validate(batch_xs,batch_ys)
            except:
                #한번에 트레이닝이 안되면 분할해서 validation한다.
                val_imgs, val_labs = divide_images(val_img , val_lab, batch_size)
                print np.shape(val_imgs[0])
                val_acc, val_loss =validate_from_images(sess,val_imgs ,val_labs ,accuracy ,cost )
                train_acc , train_loss=validate(batch_xs,batch_ys,accuracy ,cost)

            write_log(step , train_acc, train_loss , val_acc, val_loss , fp)
            softmax_=sess.run(softmax ,feed_dict ={ x:batch_xs , y_:batch_ys , keep_prob:1.0 })
            print softmax_
            np.save('./result/batch_xs' ,batch_xs)
            np.save('./result/batch_ys' ,batch_ys)
        list_aug_x=aug_8_times(batch_xs)
        for ele in list_aug_x:
            training(ele,batch_ys)

############################################aug_and_training############################
    print("--- Training Time : %s ---" % (time.time() - start_time))
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    fp.write(train_time)
    fp.close()

In [ ]:
def BATCH_TRAINING_RANDOM(maxiter, batch_size,file_locate, cost , train_step ,correct_prediction , \
                   accuracy ,softmax, fp ):
 """
 aug( O )
 
 
 """
    cost,train_step,correct_prediction , accuracy , softmax = tensor_info
    start_time = time.time()
#with tf.device('/gpu:0'):
    train_img, train_lab,val_img,val_lab,test_img,test_lab\
    =load_TVT(False,file_locate) #TVT is Train Validation Test  
    for step in range(maxiter):    
        batch_xs , batch_ys = next_batch(batch_size, train_img , train_lab)       
        if step%100 ==0: #여기 if loop에서 validate을 합니다 
            try:
                val_acc , val_loss=validate(val_img , val_lab)
                train_acc , train_loss=validate(batch_xs,batch_ys)
            except:
                #한번에 트레이닝이 안되면 분할해서 validation한다.
                val_imgs, val_labs = divide_images(val_img , val_lab, batch_size)
                print np.shape(val_imgs[0])
                val_acc, val_loss =validate_from_images(sess,val_imgs ,val_labs ,accuracy ,cost )
                train_acc , train_loss=validate(batch_xs,batch_ys,accuracy ,cost)

            write_log(step , train_acc, train_loss , val_acc, val_loss , fp)
            softmax_=sess.run(softmax ,feed_dict ={ x:batch_xs , y_:batch_ys , keep_prob:1.0 })
            print softmax_
        else
            training(batch_xs,batch_ys)

############################################aug_and_training############################
    print("--- Training Time : %s ---" % (time.time() - start_time))
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    fp.write(train_time)
    TEST(test_img, test_lab , tensor_info, batch_size ,fp)

In [2]:
def TEST( test_img, test_lab , tensor_info, batch_size ,fp):
    cost,train_step,correct_prediction , accuracy , softmax = tensor_info
    test_imgs, test_labs=divide_images(test_img, test_lab , batch_size )
    test_acc , test_loss=validate_from_images(sess,test_imgs ,test_labs ,accuracy ,cost)
    train_time="--- Training Time : ---:\t" +str(time.time() - start_time)
    
    
    print("test  accuracy %g" %(test_acc))
    print("test loss : %g" %(test_loss))
    test_str = '\ttest_loss:\t'+str(test_loss) +'\ttest accuracy:\t'+str(test_acc)+'\n'
    fp.write(test_str)
    fp.close()
    
    

In [ ]:

def write_log(step,train_acc, train_loss , val_acc , val_loss ,fp):
    """
    fp = File Pointer
    
    """
    #result = sess.run(sm_conv , feed_dict = {x:val_img , y_:batch_ys , keep_prob :1.0})
    print step
    print("step %d , training  accuracy %g" %(step,train_acc))
    print("step %d , loss : %g" %(step,train_loss))
    train_str = 'step:\t'+str(step)+'\tval_loss:\t'+str(train_loss) +'\tval accuracy:\t'+str(train_acc)+'\n'

    print("step %d , validation  accuracy %g" %(step,val_acc))
    print("step %d , validation loss : %g" %(step,val_loss))
    val_str = 'step:\t'+str(step)+'\tval_loss:\t'+str(val_loss) +'\tval accuracy:\t'+str(val_acc)+'\n'

    fp.write(train_str)
    fp.write(val_str)
    